In [1]:
import psycopg2
from imessage_reader import fetch_data
from os.path import expanduser
import sys
import sqlite3

## postgresql db 만들기
### ledger scheme에 transaction table 만들기
- transaction 테이블 만들기
- 정규화 적용
- 혹시 모르니 user_id 생성하기


In [3]:
def postgre_exe(query):
    cursor.execute(query)
    cursor.execute("COMMIT")

In [2]:
db = psycopg2.connect(host = 'localhost', dbname = 'ledger', user = 'mato', port = 5432)
cursor = db.cursor()

In [4]:
query = """CREATE SCHEMA LEDGER;"""
cursor.execute(query)
cursor.execute("COMMIT")


In [6]:
query = """CREATE TYPE WEEK AS ENUM ('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun')"""
postgre_exe(query)

In [7]:
query = """CREATE TABLE LEDGER.TRANSACTION (TRANS_ID serial PRIMARY KEY NOT NULL, TRANS_DATE DATE NOT NULL, TRANS_TIME TIME NOT NULL, DAY WEEK NOT NULL, 
            TYPE INTEGER NOT NULL, CATEGORY INTEGER NOT NULL, AMOUNTS INTEGER NOT NULL, MEMO VARCHAR(50), PLACE VARCHAR(25), PAYMENTS INTEGER NOT NULL, USER_ID INTEGER NOT NULL)"""
postgre_exe(query)

In [10]:
query = """CREATE TABLE LEDGER.TYPE (TYPE_ID INTEGER PRIMARY KEY NOT NULL, TYPE_NAME VARCHAR(20) NOT NULL);"""
postgre_exe(query)
query = """INSERT INTO LEDGER.TYPE VALUES (1, '수입'), (2,'지출');"""
postgre_exe(query)

UndefinedTable: relation "type" does not exist
LINE 1: INSERT INTO TYPE VALUES (1, '수입'), (2,'지출');
                    ^


In [12]:
query = """CREATE TABLE LEDGER.CATEGORY (CATEGORY_ID INTEGER PRIMARY KEY NOT NULL, CATEGORY_NAME VARCHAR(20) NOT NULL);"""
postgre_exe(query)
query = """INSERT INTO LEDGER.CATEGORY VALUES
            (1, '식비'), (2, '카페•간식'), (3, '편의점•마트'), (4, '술•유흥'), (5, '쇼핑'), (6, '취미•여가'), (7, '의료•건강'),
            (8, '주거•통신'), (9, '보험•세금•기타금융'), (10, '미용'), (11, '교통•자동차'), (12, '여행•숙박'), (13, '교육'),
            (14, '생활'), (15, '기부•후원'), (16, '경조사비'), (17, '선물'), (18, '기타'), (19, 'ATM출금'),
            (20, '이체'), (21, '급여'), (22, '카드대급'), (23, '저축•투자'), (24, '후불결제대금');"""
postgre_exe(query)

In [13]:
query = """CREATE TABLE LEDGER.ACCOUNTS (ACCOUNTS_ID SERIAL PRIMARY KEY NOT NULL, ACCOUNTS_BANK VARCHAR(25) NOT NULL, ACCOUNTS_NO VARCHAR(25) NOT NULL,
            ACCOUNTS_NAME VARCHAR(25) NOT NULL, ACCOUNTS_BALANCE INTEGER DEFAULT 0, ACCOUNTS_MAIN BOOLEAN DEFAULT FALSE);"""
postgre_exe(query)
query = """INSERT INTO LEDGER.ACCOUNTS (ACCOUNTS_BANK, ACCOUNTS_NO, ACCOUNTS_NAME, ACCOUNTS_MAIN) 
            VALUES ('신한은행', '110-321-894780', '주거래/급여통장', true), ('토스뱅크', '1000-1287-1132', '보조통장', false), ('신한은행', '110-324-216766', '생활비/교통비통장', false);"""
postgre_exe(query)


In [14]:
query = """CREATE TABLE LEDGER.PAYMENTS (PAYMENTS_ID SERIAL PRIMARY KEY NOT NULL, PAYMENTS_TYPE VARCHAR(25) NOT NULL, PAYMENTS_NAME VARCHAR(25),
            PAYMENTS_MAIN BOOLEAN DEFAULT false, ACCOUNTS_ID INTEGER, REMAKRS VARCHAR(25),
            CONSTRAINT FK_ACCOUNTS FOREIGN KEY(ACCOUNTS_ID) REFERENCES LEDGER.ACCOUNTS(ACCOUNTS_ID) ON DELETE CASCADE);"""
postgre_exe(query)


In [15]:
query = """INSERT INTO LEDGER.PAYMENTS (PAYMENTS_TYPE, PAYMENTS_NAME, PAYMENTS_MAIN, ACCOUNTS_ID) VALUES 
            ('체크카드', '신한헤이영', true, 1), ('체크카드', '토스', false, 2);"""
postgre_exe(query)

In [19]:
query = """CREATE TABLE LEDGER.TRANSACTION (TRANS_ID serial PRIMARY KEY NOT NULL, TRANS_DATE DATE NOT NULL, TRANS_TIME TIME NOT NULL, DAY WEEK NOT NULL, 
            TYPE INTEGER NOT NULL, CATEGORY INTEGER NOT NULL, AMOUNTS INTEGER NOT NULL, MEMO VARCHAR(50), PLACE VARCHAR(25), PAYMENTS INTEGER NOT NULL,
            CONSTRAINT FK_TYPE FOREIGN KEY(TYPE) REFERENCES LEDGER.TYPE(TYPE_ID) ON DELETE CASCADE,
            CONSTRAINT FK_CATEGORY FOREIGN KEY(CATEGORY) REFERENCES LEDGER.CATEGORY(CATEGORY_ID) ON DELETE CASCADE,
            CONSTRAINT FK_PAYMENTS FOREIGN KEY(PAYMENTS) REFERENCES LEDGER.PAYMENTS(PAYMENTS_ID) ON DELETE CASCADE);"""
postgre_exe(query)

### message sheme에 문자 데이터 디비 만들기

In [20]:
def fetch_db_data(db, command):
    try:
        conn = sqlite3.connect(db)
        cursor = conn.cursor()
        cursor.execute(command)
        return cursor.fetchall()
    except Exception as e:
        sys.exit("Error reading the database: %s" %e)

In [21]:
db_path = expanduser("~") + "/Library/Messages/chat.db"
query = """SELECT text, handle.id, datetime((date / 1000000000) + 978307200, 'unixepoch', 'localtime'),
handle.service, message.destination_caller_id, message.is_from_me, date((date/1000000000)+978307200, 'unixepoch', 'localtime')
FROM message JOIN handle ON message.handle_id = handle.ROWID
WHERE handle.id == '+8215447200' and message.is_from_me == 0 and date((date/1000000000)+978307200, 'unixepoch', 'localtime') >= '2023-01-01'
ORDER BY date desc"""

In [22]:
rval = fetch_db_data(db_path, query)
rval

[('[Web발신]\n[신한체크승인] 박*준(2709) 02/08 11:35 7,000원 베니스F&S 이레 잔액16,202원',
  '+8215447200',
  '2023-02-08 11:35:11',
  'SMS',
  '+821055375278',
  0,
  '2023-02-08'),
 ('[Web발신]\n[신한체크승인] 박*준(2709) 02/08 08:23 1,500원 씨유당산중앙점 잔액23,202원',
  '+8215447200',
  '2023-02-08 08:23:09',
  'SMS',
  '+821055375278',
  0,
  '2023-02-08'),
 ('[Web발신]\n[신한체크승인] 박*준(2709) 02/07 22:27 6,300원 티머니 개인택  잔액24,702원',
  '+8215447200',
  '2023-02-07 22:27:57',
  'SMS',
  '+821055375278',
  0,
  '2023-02-07'),
 ('[Web발신]\n[신한체크승인] 박*준(2709) 02/07 18:39 2,700원 지에스(GS)25   잔액31,002원',
  '+8215447200',
  '2023-02-07 18:39:21',
  'SMS',
  '+821055375278',
  0,
  '2023-02-07'),
 ('[Web발신]\n[신한체크승인] 박*준(2709) 02/07 11:33 7,000원 베니스F&S 이레 잔액33,702원',
  '+8215447200',
  '2023-02-07 11:33:27',
  'SMS',
  '+821055375278',
  0,
  '2023-02-07'),
 ('[Web발신]\n[신한체크승인] 박*준(2709) 02/07 08:22 1,600원 씨유당산중앙점 잔액40,702원',
  '+8215447200',
  '2023-02-07 08:22:23',
  'SMS',
  '+821055375278',
  0,
  '2023-02-07'),
 ('[Web발신]\n[신한체크승인

In [ ]:
query = 

In [23]:
db = psycopg2.connect(host = 'localhost', dbname = 'ledger', user = 'mato', port = 5432)
cursor = db.cursor()

In [24]:
query = """CREATE SCHEMA MESSAGE;"""
postgre_exe(query)


SyntaxError: syntax error at or near "PRUMARY"
LINE 1: CREATE TABLE MESSAGE.MESSAGE_ID (ID SERIAL PRUMARY KEY NOT N...
                                                   ^


In [26]:
query = """CREATE TABLE MESSAGE.MESSAGE_ID (ID SERIAL PRIMARY KEY NOT NULL, NUMBER VARCHAR(25) NOT NULL, NAME VARCHAR(25));"""
postgre_exe(query)


NumericValueOutOfRange: value "+8215447200" is out of range for type integer
LINE 1: ...SERT INTO MESSAGE.MESSAGE_ID (ID, NUMBER) VALUES ('+82154472...
                                                             ^


In [27]:
query = """INSERT INTO MESSAGE.MESSAGE_ID ( NUMBER, NAME) VALUES ('+8215447200', '신한헤이영');"""
postgre_exe(query)

In [30]:
query = """ALTER TABLE MESSAGE.MESSAGE_ID ADD CONSTRAINT UNIQ_NUMBER UNIQUE(NUMBER);"""
postgre_exe(query)

In [31]:
query = """CREATE TABLE MESSAGE.TEXT_MESSAGE (ID serial PRIMARY KEY NOT NULL, NUMBER varchar(25) NOT NULL, 
TEXT_ORIGINAL varchar(255) NOT NULL, DATE DATE NOT NULL, TIME TIME NOT NULL, AMMOUNTS INTEGER NOT NULL, PLACE VARCHAR(25) NOT NULL,
SACE_CHECK boolean default false,
CONSTRAINT FK_NUMBER FOREIGN KEY(NUMBER) REFERENCES MESSAGE.MESSAGE_ID(NUMBER) ON DELETE CASCADE)"""
postgre_exe(query)

In [32]:
for i in rval:
    try:
        number = i[1]
        text_original = i[0]
        date = i[6]
        time = i[0].split(" ")[3]
        ammounts = int(i[0].split(" ")[4][:-1].replace(',', ''))
        place = i[0].split(" ")[5]
        query = f"""INSERT INTO message.text_message (number, text_original, date, time, ammounts, place) 
                    VALUES('{number}', '{text_original}', '{date}', '{time}', {ammounts}, '{place}')"""
        postgre_exe(query)
#         print( 'date: ', i[6])
#         print( 'time: ', i[0].split(" ")[3])
#         print( 'pay: ', int(i[0].split(" ")[4][:-1].replace(',', '')))
#         print( 'place: ', i[0].split(" ")[5])
        print("Success!")
        print(' ======================== ')
    except Exception as e:
        print(e)

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S